Build a neural network<br>
Neural networks are comprised of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.
<br>
In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [3]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get a hardware device for training<br>
We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let's check to see if torch.cuda is available, else we continue to use the CPU.

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Define the class<br>
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU())

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print it's structure.

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's forward, along with some background operations. However, do not call model.forward() directly! Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
<br>
We get the prediction densities by passing it through an instance of the nn.Softmax.

In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn. Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


Weight and Bias<br>
the nn.Linear module randomly initializes the weights and bias for each layer and internally stores the values in Tnsors.

In [11]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")
print(f"First Linear weights: {model.linear_relu_stack[0].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0283, -0.0243,  0.0049,  ...,  0.0151,  0.0300,  0.0153],
        [-0.0118, -0.0102,  0.0020,  ...,  0.0292, -0.0283,  0.0140],
        [-0.0027, -0.0154, -0.0096,  ...,  0.0027,  0.0249, -0.0152],
        ...,
        [-0.0032,  0.0257, -0.0109,  ...,  0.0089, -0.0026,  0.0116],
        [-0.0144, -0.0341,  0.0143,  ..., -0.0052,  0.0210,  0.0174],
        [-0.0021, -0.0290, -0.0313,  ...,  0.0309,  0.0064, -0.0341]],
       device='cuda:0', requires_grad=True) 

First Linear weights: Parameter containing:
tensor([-1.9540e-02,  2.8482e-02, -2.2956e-02,  2.5327e-02, -1.8137e-02,
         1.3479e-02,  6.6526e-03, -4.3927e-03,  2.8558e-03, -7.6080e-03,
         2.4561e-02, -2.0997e-03,  2.7080e-02,  1.4545e-02, -1.4017e-02,
         1.9700e-02,  1.5289e-02, -4.0830e-03, -8.1522e-03,  1.5783e-02,
         1.8770e-02, -1.2264e-02,  9.1846e-03,  2.2370e-02,  1.0805e-02,
         3.1949e-02,  1.7185e-02,  8.0019e-03, -1.2449e-02,  2.5554

Model layers <br>
Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])



nn.Flatten<br>
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained). Each of the pixels are pass to the input layer of the neural network.

In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear<br>
The linear layer is a module that applies a linear transformation on the input using it's stored weights and biases. The gayscale value of each pixel in the input layer will be connected to neurons in the hidden layer for calculation. The calculation used for the transformation is weight*input + bias

In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

In [15]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0462, -0.3010,  0.5002,  0.1128, -0.0976,  0.4411, -0.0614, -0.3842,
          0.2014,  0.0769, -0.0272,  0.5066, -0.9085, -0.0308,  0.0757,  0.2414,
          0.7776,  0.3511, -0.6870,  0.2679],
        [ 0.2838, -0.3519,  0.2310,  0.1383, -0.0072,  0.4355, -0.4603, -0.6335,
         -0.4013, -0.1066,  0.3098,  0.7858, -0.5705, -0.3388, -0.1133,  0.1542,
          0.3493,  0.3599, -0.7445, -0.0070],
        [ 0.4607, -0.5238,  0.4081,  0.4793,  0.2885,  0.3448, -0.0588, -0.3215,
         -0.5881, -0.1561, -0.0986,  0.5996, -0.5078, -0.2746, -0.2765,  0.1627,
          0.2558,  0.4529, -0.6496,  0.3061]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0462, 0.0000, 0.5002, 0.1128, 0.0000, 0.4411, 0.0000, 0.0000, 0.2014,
         0.0769, 0.0000, 0.5066, 0.0000, 0.0000, 0.0757, 0.2414, 0.7776, 0.3511,
         0.0000, 0.2679],
        [0.2838, 0.0000, 0.2310, 0.1383, 0.0000, 0.4355, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3098, 0.7858, 0.0000, 0.0000, 0.00

nn.Sequential

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax

In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model parameters<br>
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's parameters() or named_parameters() methods.
<br>
In this example, we iterate over each parameter, and print its size and a preview of its values.

In [18]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0283, -0.0243,  0.0049,  ...,  0.0151,  0.0300,  0.0153],
        [-0.0118, -0.0102,  0.0020,  ...,  0.0292, -0.0283,  0.0140]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0195,  0.0285], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0262, -0.0017, -0.0014,  ..., -0.0014,  0.0066, -0.0134],
        [-0.0403, -0.0439, -0.0211,  ...,  0.0435, -0.0117,  0.0051]],
       device='cu